# ALBERT를 이용한 감정 분석
사전학습모델 : albert-base-v2<br>
데이터 : GLUE_SST-2 (Stanford Sentiment Treebank v2)

# 사전 준비

In [ ]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

**GLUE의 SST-2 데이터 불러오기**

In [2]:
from datasets import load_dataset

datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [4]:
# label 0: negative(부정)) / 1: positive(긍정) / -1: test data
print(datasets["train"][0])
print(datasets["validation"][0])
print(datasets["test"][0])

{'sentence': 'hide new secretions from the parental units ', 'label': 0, 'idx': 0}
{'sentence': "it 's a charming and often affecting journey . ", 'label': 1, 'idx': 0}
{'sentence': 'uneasy mishmash of styles and genres .', 'label': -1, 'idx': 0}


**ALBERT 모델과 토크나이저 불러오기**

In [5]:
from transformers import AlbertForSequenceClassification, AlbertTokenizer

tokenizer = AlbertTokenizer.from_pretrained("albert-base-v2")
albert_model = AlbertForSequenceClassification.from_pretrained("albert-base-v2")

Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.decoder.bias', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.bias', 'predictions.LayerNorm.weight', 'predictions.LayerNorm.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.weight', 'classifier.bias']
You sho

In [6]:
albert_model.config

AlbertConfig {
  "_name_or_path": "albert-base-v2",
  "architectures": [
    "AlbertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0,
  "bos_token_id": 2,
  "classifier_dropout_prob": 0.1,
  "down_scale_factor": 1,
  "embedding_size": 128,
  "eos_token_id": 3,
  "gap_size": 0,
  "hidden_act": "gelu_new",
  "hidden_dropout_prob": 0,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inner_group_num": 1,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "albert",
  "net_structure_type": 0,
  "num_attention_heads": 12,
  "num_hidden_groups": 1,
  "num_hidden_layers": 12,
  "num_memory_blocks": 0,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.21.0",
  "type_vocab_size": 2,
  "vocab_size": 30000
}

# 토크나이징, 데이터 구축

**데이터 준비**

In [7]:
from tqdm import tqdm

In [8]:
# train data가 약 67000개로 너무 많아 10000개로 줄임
train_sentence = [datasets['train']['sentence'][idx] for idx in tqdm(range(0, 10000))]
train_label = [datasets['train']['label'][idx] for idx in tqdm(range(0, 10000))]

100%|██████████| 10000/10000 [03:52<00:00, 42.95it/s]


In [9]:
ids = datasets['validation'].num_rows
val_sentence = [datasets['validation']['sentence'][idx] for idx in range(0, ids)]
val_label = [datasets['validation']['label'][idx] for idx in range(0, ids)]

In [10]:
ids = datasets['test'].num_rows
test_sentence = [datasets['test']['sentence'][idx] for idx in range(0, ids)]
test_label = [datasets['test']['label'][idx] for idx in range(0, ids)]

In [11]:
# ALBERT모델에서 정한 입력 크기보다 크면 잘라내기, 패딩 채우기
train_input = tokenizer(train_sentence, truncation=True, padding=True, return_tensors="pt")
val_input = tokenizer(val_sentence, truncation=True, padding=True, return_tensors="pt")
test_input = tokenizer(test_sentence, truncation=True, padding=True, return_tensors="pt")

**데이터셋 변환**

In [12]:
import torch

In [13]:
class SSTDataset(torch.utils.data.Dataset):
    def __init__(self, inputs, labels):
        self.inputs = inputs
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.inputs.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = SSTDataset(train_input, train_label)
val_dataset = SSTDataset(val_input, val_label)
test_dataset = SSTDataset(test_input, test_label)